In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
import numpy as np

# Set a random seed to make random operations reproducible
random_seed = 42  # You can choose any integer value as the seed

# Set the random seed for NumPy
np.random.seed(random_seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, concatenate
from keras.optimizers import Adam
from keras import backend as K


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import *
from keras import initializers
from keras.layers import BatchNormalization

In [ ]:
print("Start load train set")
# Load and preprocess training data
imgs_train = np.load('/content/drive/MyDrive/LUNA16/trainImages.npy').astype(np.float16)
imgs_mask_train = np.load('/content/drive/MyDrive/LUNA16/trainMasks.npy').astype(np.float16)
print("Finish load train set")


print("Start tranpose train set")
imgs_train = imgs_train.transpose(0, 2, 3, 1)
imgs_mask_train = imgs_mask_train.transpose(0, 2, 3, 1)
imgs_mask_train[imgs_mask_train > 0.] = 1.0
print("Finish tranpose load train set")



Start load train set
Finish load train set
Start tranpose train set
Finish tranpose load train set


In [ ]:

class_weights = {0: 0.9740012017916528, 1: 0.02599879820834716}
print("Class Weights:", class_weights)


Class Weights: {0: 0.9740012017916528, 1: 0.02599879820834716}


In [ ]:
from sklearn.model_selection import train_test_split
# Split the training data into training and validation sets
imgs_train, val_images, imgs_mask_train, val_masks = train_test_split(imgs_train, imgs_mask_train, test_size=0.2, random_state=42)


In [ ]:
def data_generator(imgs, masks, batch_size=1):
    num_samples = len(imgs)
    if random_seed is not None:
        np.random.seed(random_seed)  # Set the random seed

    while True:
        indices = np.random.choice(num_samples, batch_size, replace= False)
        batch_imgs = imgs[indices]
        batch_masks = masks[indices]
        yield batch_imgs, batch_masks

In [ ]:
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    smooth = 0.
    intersection = K.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (K.sum(y_true) + K.sum(y_pred) + smooth)

def dice_coefficient(y_true, y_pred):
    smooth = 1.0

    # Flatten the true and predicted masks
    y_true_flat = tf.keras.backend.flatten(y_true)
    y_pred_flat = tf.keras.backend.flatten(y_pred)

    intersection = tf.keras.backend.sum(y_true_flat * y_pred_flat)
    dice_coeff_class_0 = (2.0 * intersection + smooth) / (tf.keras.backend.sum(y_true_flat) + tf.keras.backend.sum(y_pred_flat) + smooth)

    y_true_class_1 = 1 - y_true
    y_pred_class_1 = 1 - y_pred

    y_true_flat_class_1 = tf.keras.backend.flatten(y_true_class_1)
    y_pred_flat_class_1 = tf.keras.backend.flatten(y_pred_class_1)

    intersection_class_1 = tf.keras.backend.sum(y_true_flat_class_1 * y_pred_flat_class_1)
    dice_coeff_class_1 = (2.0 * intersection_class_1 + smooth) / (tf.keras.backend.sum(y_true_flat_class_1) + tf.keras.backend.sum(y_pred_flat_class_1) + smooth)

    # Calculate the mean of both Dice coefficients
    dice_coefficient = (dice_coeff_class_0 + dice_coeff_class_1) / 2.0
    return dice_coefficient

def weighted_dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
def weighted_binary_crossentropy(y_true, y_pred):
    weight_positive = 0.98  # Set a suitable weight for the positive class (you can experiment with different values)
    weight_negative = 0.02  # Set a suitable weight for the negative class (you can experiment with different values)

    # Calculate the binary cross-entropy loss
    bce = K.binary_crossentropy(y_true, y_pred)

    # Apply the weights to the loss based on the true labels
    weighted_bce = tf.where(K.equal(y_true, 1), weight_positive * bce, weight_negative * bce)

    return K.mean(weighted_bce)

In [ ]:

def get_unet_small():
    inputs = Input((512,512,1))
    conv1 = Conv2D(32, (3, 3), activation='elu', padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='elu',padding='same', name='conv_1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), name='pool_1')(conv1)
    pool1 = BatchNormalization()(pool1)

    conv2 = Conv2D(64, (3, 3), activation='elu',padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='elu',padding='same', name='conv_2')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), name='pool_2')(conv2)
    pool2 = BatchNormalization()(pool2)

    conv3 = Conv2D(128, (3, 3), activation='elu',padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='elu',padding='same', name='conv_3')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), name='pool_3')(conv3)
    pool3 = BatchNormalization()(pool3)

    conv4 = Conv2D(256, (3, 3), activation='elu',padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='elu',padding='same', name='conv_4')(conv4)
    conv4 = BatchNormalization()(conv4)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3],axis = 3)
    conv7 = Conv2D(128, (3, 3), activation='elu',padding='same')(up7)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(128, (3, 3), activation='elu',padding='same', name='conv_7')(conv7)
    conv7 = BatchNormalization()(conv7)

    #up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2],axis = 3)
    conv8 = Conv2D(64, (3, 3), activation='elu',padding='same')(up8)
    conv8 = Dropout(0.2)(conv8)
    conv8 = Conv2D(64, (3, 3), activation='elu',padding='same', name='conv_8')(conv8)
    conv8 = BatchNormalization()(conv8)

    #up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1],axis = 3)
    conv9 = Conv2D(32, (3, 3), activation='elu',padding='same')(up9)
    conv9 = Dropout(0.2)(conv9)
    conv9 = Conv2D(32, (3, 3), activation='elu',padding='same', name='conv_9')(conv9)
    conv9 = BatchNormalization()(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid', name='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    # Compile the model with the custom loss function and optimizer
    model.summary()

    return model


In [ ]:
num_samples_subset = 100  # Choose an appropriate number of samples
imgs_train_subset = imgs_train[:num_samples_subset]
imgs_mask_train_subset = imgs_mask_train[:num_samples_subset]

print("Number sample : " ,  num_samples_subset)
print("Number sample : " ,  imgs_train_subset.shape)
print("Number sample : " ,  imgs_mask_train_subset.shape)


Number sample :  100
Number sample :  (100, 512, 512, 1)
Number sample :  (100, 512, 512, 1)


In [ ]:
num_samples_subset = 50  # Choose an appropriate number of samples
val_images= val_images[:num_samples_subset]
val_masks = val_masks[:num_samples_subset]

print("Number sample : " ,  num_samples_subset)
print("Number sample : " ,  val_images.shape)
print("Number sample : " ,  val_masks.shape)


Number sample :  50
Number sample :  (50, 512, 512, 1)
Number sample :  (50, 512, 512, 1)


In [ ]:
img_train_tuning = imgs_train_subset.copy()
print(img_train_tuning.shape)

imgs_mask_train_subset_tuning = imgs_mask_train_subset.copy()
print(imgs_mask_train_subset_tuning.shape)
val_images_tuning = val_images.copy()
val_masks_tuning = val_masks.copy()


(100, 512, 512, 1)
(100, 512, 512, 1)


In [ ]:

model = get_unet_small()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 32)         320       ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 512, 512, 32)         0         ['conv2d[0][0]']              
                                                                                                  
 conv_1 (Conv2D)             (None, 512, 512, 32)         9248      ['dropout[0][0]']             
                                                                                              

In [ ]:

from keras.optimizers import Adam
import tensorflow as tf
import optuna
from optuna.pruners import MedianPruner
from sklearn.model_selection import StratifiedKFold

import numpy as np

import tensorflow as tf



def create_and_train_model( steps_per_epoch, epochs, validation_data, learning_rate):
    # Clear any previously allocated memory
    tf.keras.backend.clear_session()

    # Set mixed-precision policy if supported by GPU
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)

    optimizer = Adam(learning_rate=learning_rate)  # Adjust the optimizer and learning rate as needed
    model.compile(optimizer=optimizer, loss=weighted_dice_coef_loss, metrics=['accuracy', dice_coefficient])

    # Train the model
    model.fit(imgs_train_subset ,imgs_mask_train_subset, steps_per_epoch=steps_per_epoch, epochs=epochs, validation_data=validation_data)

    return model


def objective(trial):
    # Define the hyperparameter search space
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)


    # Create a new data generator instance for each trial
    steps_per_epoch = len(imgs_train_subset)
    validation_data = (val_images, val_masks)

    # Create and train the model with the suggested hyperparameters
    model = create_and_train_model(steps_per_epoch, 1, validation_data, learning_rate=learning_rate)

    # Evaluate the model on the test set
    metric_name = 'dice_coefficient'
    score = model.evaluate(validation_data[0], validation_data[1], verbose=0)
    metric_value = score[model.metrics_names.index(metric_name)]

    return metric_value


# Create a StratifiedKFold object for cross-validation (if needed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)



# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize', pruner=MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials = 15, timeout=5400)


# Get the best hyperparameters and best score
best_params = study.best_params
best_score = study.best_value
print(best_params)
print(best_score)


[I 2023-09-12 13:09:44,215] A new study created in memory with name: no-name-927f2370-b045-4b42-8875-51ad4bf43542
<ipython-input-16-de3eaae303e5>:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)


100/100 [==============================] - 76s 531ms/step - loss: 0.9022 - accuracy: 0.5904 - dice_coefficient: 0.4146 - val_loss: 0.9389 - val_accuracy: 0.1632 - val_dice_coefficient: 0.1515


[I 2023-09-12 13:11:04,148] Trial 0 finished with value: 0.15154191851615906 and parameters: {'learning_rate': 0.0006217288069704359}. Best is trial 0 with value: 0.15154191851615906.


100/100 [==============================] - 22s 131ms/step - loss: 0.8941 - accuracy: 0.6529 - dice_coefficient: 0.4426 - val_loss: 0.9123 - val_accuracy: 0.4448 - val_dice_coefficient: 0.3437


[I 2023-09-12 13:11:31,581] Trial 1 finished with value: 0.3436918556690216 and parameters: {'learning_rate': 0.0018546526712894685}. Best is trial 1 with value: 0.3436918556690216.


...

100/100 [==============================] - 23s 142ms/step - loss: 0.7267 - accuracy: 0.9408 - dice_coefficient: 0.6211 - val_loss: 0.8416 - val_accuracy: 0.8664 - val_dice_coefficient: 0.5426


[I 2023-09-12 13:16:55,935] Trial 13 finished with value: 0.5425677299499512 and parameters: {'learning_rate': 0.0003603966186357009}. Best is trial 6 with value: 0.5674395561218262.


100/100 [==============================] - 23s 142ms/step - loss: 0.7137 - accuracy: 0.9431 - dice_coefficient: 0.6282 - val_loss: 0.8386 - val_accuracy: 0.8610 - val_dice_coefficient: 0.5422


[I 2023-09-12 13:17:23,218] Trial 14 finished with value: 0.5421510934829712 and parameters: {'learning_rate': 0.00010429030087527029}. Best is trial 6 with value: 0.5674395561218262.


{'learning_rate': 0.00016815389884367667}
0.5674395561218262


In [ ]:
from keras.optimizers import Adam
import tensorflow as tf
import optuna
from optuna.pruners import MedianPruner
from sklearn.model_selection import StratifiedKFold

import numpy as np

import tensorflow as tf



def create_and_train_model( steps_per_epoch, epochs, validation_data,batch_size):
    # Clear any previously allocated memory
    tf.keras.backend.clear_session()

    # Set mixed-precision policy if supported by GPU
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)

    optimizer = Adam(learning_rate = 0.00016815389884367667)  # Adjust the optimizer and learning rate as needed
    model.compile(optimizer=optimizer, loss=weighted_dice_coef_loss, metrics=['accuracy', dice_coefficient])

    # Train the model
    model.fit(imgs_train_subset ,imgs_mask_train_subset, batch_size = batch_size, epochs=epochs, validation_data=validation_data)

    return model


def objective(trial):
    # Define the hyperparameter search space
    batch_size = int(trial.suggest_loguniform('batch size', 1 , 12))


    # Create a new data generator instance for each trial
    steps_per_epoch = len(imgs_train_subset)
    validation_data = (val_images, val_masks)

    # Create and train the model with the suggested hyperparameters
    model = create_and_train_model(steps_per_epoch, 1, validation_data, batch_size )

    # Evaluate the model on the test set
    metric_name = 'dice_coefficient'
    score = model.evaluate(validation_data[0], validation_data[1], verbose=0)
    metric_value = score[model.metrics_names.index(metric_name)]

    return metric_value


# Create a StratifiedKFold object for cross-validation (if needed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)



# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize', pruner=MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials = 15, timeout=5400)


# Get the best hyperparameters and best score
best_params = study.best_params
best_score = study.best_value
print(best_params)
print(best_score)


[I 2023-09-13 13:17:23,340] A new study created in memory with name: no-name-4aac24dc-eab1-4978-9dbb-241a235c1d0e
<ipython-input-17-182bce2440cf>:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  batch_size = int(trial.suggest_loguniform('batch size', 1 , 12))


20/20 [==============================] - 44s 702ms/step - loss: 0.9102 - accuracy: 0.5682 - dice_coefficient: 0.3950 - val_loss: 0.9021 - val_accuracy: 0.6269 - val_dice_coefficient: 0.4293


[I 2023-09-13 13:18:51,073] Trial 0 finished with value: 0.4270666241645813 and parameters: {'batch size': 5.0967228397942}. Best is trial 0 with value: 0.4270666241645813.


100/100 [==============================] - 25s 137ms/step - loss: 0.8794 - accuracy: 0.7188 - dice_coefficient: 0.4638 - val_loss: 0.9468 - val_accuracy: 0.0286 - val_dice_coefficient: 0.0278


[I 2023-09-13 13:19:37,740] Trial 1 finished with value: 0.028019249439239502 and parameters: {'batch size': 1.5786425118272096}. Best is trial 0 with value: 0.4270666241645813.


...

100/100 [==============================] - 20s 133ms/step - loss: 0.7587 - accuracy: 0.9017 - dice_coefficient: 0.5848 - val_loss: 0.8828 - val_accuracy: 0.6085 - val_dice_coefficient: 0.4274


[I 2023-09-13 13:28:02,372] Trial 13 finished with value: 0.42336928844451904 and parameters: {'batch size': 1.0433025016102957}. Best is trial 11 with value: 0.5515937805175781.


34/34 [==============================] - 20s 383ms/step - loss: 0.7227 - accuracy: 0.9205 - dice_coefficient: 0.6082 - val_loss: 0.7537 - val_accuracy: 0.8877 - val_dice_coefficient: 0.5834


[I 2023-09-13 13:28:29,537] Trial 14 finished with value: 0.5820558071136475 and parameters: {'batch size': 3.89779746018928}. Best is trial 14 with value: 0.5820558071136475.


{'batch size': 3.89779746018928}
0.5820558071136475
